In [3]:
pip install sentencepiece

In [1]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

In [3]:
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"id": "main_content"})
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    summary = summarization_pipeline(text_content, max_length=150, min_length=30, do_sample=False)
    
    # Return the summary as a string
    return summary[0]['summary_text']

url = 'https://indiankanoon.org/doc/168100369/'
summary = summarize_legal_document(url)
print(summary)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [6]:
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"id": "main_content"})
    if main_content is None:
        return "Error: Could not find main content"
    
    paragraphs = main_content.find_all('p')
    if not paragraphs:
        return "Error: Could not find any paragraphs in main content"
    
    text_content = ' '.join([p.text for p in paragraphs])
    
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    summary = summarization_pipeline(text_content, max_length=150000, min_length=30, do_sample=False)
    
    # Return the summary as a string
    return summary[0]['summary_text']
url = 'https://indiankanoon.org/doc/168100369/'
summary = summarize_legal_document(url)
print(summary)

Error: Could not find main content


We got error till now as there is no div with id main_content

In [10]:
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content  # <-- Add this line
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    paragraphs = main_content.find_all('p')
    print(paragraphs)
    text_content = ' '.join([p.text for p in paragraphs])
    
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    summary = summarization_pipeline(text_content, max_length=150, min_length=30, do_sample=False)
    
    # Return the summary as a string
    return summary[0]['summary_text']


url = 'https://indiankanoon.org/doc/168100369/'
summary = summarize_legal_document(url)
print(summary)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[<p>Take notes as you read a judgment using our <b>Virtual Legal Assistant</b> and get email alerts whenever a new judgment matches your query (<b>Query Alert Service</b>). Try out our <b>Premium Member</b> services -- <a href="/members/">Free for one month</a>.</p>, <p id="p_1">           1.    Leave granted.
</p>, <p id="p_2">           2.    This appeal has been  filed  impugning  the  judgment  and
           order dated 12th November,  2010  in  FAO(OS)  NO.607  of  2010,
           whereby the Division Bench of the Delhi High Court dismissed the
           appeal filed by the appellant in I.A.  NO.13902  of  2008  filed
           under Order VIII rule 1 praying for extension of time for filing
           written statement by the defendant  i.e.  the  appellant  herein
           till I.A. No. 11803 of 2008 filed  under  Order  I  rule  10  to
           implead Mr. Sharad Maheshwari as plaintiff.  The  aforesaid  Mr.
           Sharad Maheshwari is the  Managing  Director  of  t

Token indices sequence length is longer than the specified maximum sequence length for this model (2389 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

The above model is not able to handle documents of such a large size

In [1]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

def summarize_legal_document(url, max_length=800, min_length=50, num_beams=4):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    
    # Load the summarization model and tokenizer
    model_name = "sshleifer/distilbart-xsum-12-1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Tokenize the input text and generate the summary
    inputs = tokenizer.encode(text_content, return_tensors='pt', max_length=max_length, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, num_beams=num_beams)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Return the summary as a string
    return summary

url = 'https://indiankanoon.org/doc/105912122/'
summary = summarize_legal_document(url)
print(summary)


 The Supreme Court in India has heard a petition calling for changes to the way the way it deals with the dishonor of cheques in the country, which are still being held in a case against a number of cases involving involving banks and banks.


It generated the summary but not too proper

# Considering document length for setting max_length hyperparameter
# Chatgpt answer differently

In [2]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    document_length = len(text_content)
    if document_length < 1000:
        max_length = 100
    elif document_length < 2000:
        max_length = 150
    elif document_length < 3000:
        max_length = 200
    else:
        max_length = 250
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    try:
        summary = summarization_pipeline(text_content, max_length=max_length, min_length=30, do_sample=False, truncation=True)
    except ValueError as e:
        return f"Error: {e}"
    
    # Return the summary as a string
    return summary[0]['summary_text']

# Example usage
url = 'https://indiankanoon.org/doc/189173396/'
summary = summarize_legal_document(url)
print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Dipak Misra, J.R. wrote a writ of appeal against Hindustan Coca-Coca-Cola Beverages Pvt. Ltd., P.O. R.R., Jorhat . The High Court of Gauhati upheld the validity of the Finance Act, 2003 . The case was brought before the High Court in a bid to address the retrospective application of the application as the same application did not address the same .


We considered using the default model with static and dynamic max_length

In [26]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    try:
        summary = summarization_pipeline(text_content, max_length=512, min_length=30, do_sample=False, truncation=True)
    except ValueError as e:
        return f"Error: {e}"
    
    # Return the summary as a string
    return summary[0]['summary_text']

# Example usage
url = 'https://indiankanoon.org/doc/105912122/'
summary = summarize_legal_document(url)
print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Indian Banks’ Association (IBA) along with Punjab . National Bank and another, seeking the following reliefs :- Laying down appropriate guidelines/directions to be followed  by all . Courts within the territory of India competent to try  a complaint. under Section 138 of the Negotiable Instruments Act, 1881 . Issue a writ of mandamus directing  the  respondents,  to adopt  necessary policy and legislative changes to deal with cases relating to dishonor of cheques .


In [28]:
url = 'https://indiankanoon.org/doc/1072049/'
summary = summarize_legal_document(url)
print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Appeal from the High Court of Judicature, Bombay, in a reference under section 66 of the Indian Income-tax . The company had paid Rs. 1,22,675 as property tax and Rs. 32,760 as urban property tax . The High Court answered all three questions in the negative and hence this appeal .


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    document_length = len(text_content)
    if document_length < 1000:
        max_length = 100
    elif document_length < 2000:
        max_length = 150
    elif document_length < 3000:
        max_length = 200
    else:
        max_length = 250
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    try:
        summary = summarization_pipeline(text_content, max_length=max_length, min_length=30, do_sample=False, truncation=True)
    except ValueError as e:
        return f"Error: {e}"
    
    # Return the summary as a string
    return summary[0]['summary_text']

# Example usage
path = 'Case Links-20230417T110231Z-001/Case Links/'
file = open(path+'1971-1980/urls_71_80.txt','r')
df = pd.DataFrame(columns=['Case Number','Summary'])
count=0
for url in file:
    summary = summarize_legal_document(url.strip())
    case_num = url.split('/')[-2]
#     file = open(path+'1947/'+case_num+'.txt','w')
#     file.write(summary)
#     file.close()
    df = pd.concat([pd.DataFrame([[case_num,summary]],columns=df.columns), df], ignore_index=True)
    df.to_csv(path+'1971-1980/summary_71_80.csv',index=False)
    count = count+1
    if count%10 == 0:
        print(count)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

10


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

20


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

...

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

3100


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

3110


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


It is taking around 20 min to summarize 100 documents

# nsi319/legal-led-base-16384 - HugginFace model gives summary in more legal-domain specific language and also repeated sentences, so we are using a default model (sshleifer/distilbart-cnn-12-6) which is giving summaries which even a layman can understand and also not repeating the sentences
# The legal-led-base-16384 model was trained on sec-litigation-releases (US) dataset consisting more than 2700 litigation releases and complaints. So it is adding some extra points related to some US cases in the summary even if the document context is Indian

In [8]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")

padding = "max_length"
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    input_tokenized = tokenizer.encode(text_content, return_tensors='pt',padding=padding,pad_to_max_length=True, max_length=6144,truncation=True)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=4,
                                  no_repeat_ngram_size=3,
                                  length_penalty=3,
                                  min_length=350,
                                  max_length=1000)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    return summary
    
# Example usage
path = 'Case Links-20230417T110231Z-001/Case Links/'
file = open(path+'1947/urls_1947.txt','r')
df = pd.DataFrame(columns=['Case Number','Summary'])
count=0
# for url in file:
url = 'https://indiankanoon.org/doc/265262/'
summary = summarize_legal_document(url.strip())
print(summary)
#     break
#     case_num = url.split('/')[-2]
#     file = open(path+'1947/'+case_num+'.txt','w')
#     file.write(summary)
#     file.close()
#     df = pd.concat([pd.DataFrame([[case_num,summary]],columns=df.columns), df], ignore_index=True)
#     df.to_csv(path+'1947/summary_1947.csv',index=False)
#     count = count+1
#     if count%10 == 0:
#         print(count)


The Honorable D. G. Palakar of the United States District Court for the Southern District of New York today announced that it has entered a final judgment against a proposed land reforms law.  According to the SEC's complaint, filed in the U.S. District Court in New York, on April 15, 1967, the Punjab and Haryana High Court entered final judgments against two land reform defendants, Amar Singh and Indraj Singh.  The SEC alleges that the defendants violated Section 10(b) of the Securities and Exchange Act of 1934 and Rule 10b-5 thereunder, and that they violated Section 17(a)(1) and (3) of that Act.  In addition, the court found that, contrary to the representations made to the court, the defendants were not entitled to the right to evict tenants from the surplus areas.  As alleged, these defendants were entitled to a minimum of ten years' tenancy, which was deemed sufficient for the tenants to be entitled to re-settlement in the newly-created "reserved area."  The court also ordered th

In [37]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")

padding = "max_length"
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    input_tokenized = tokenizer.encode(text_content, return_tensors='pt',padding=padding,pad_to_max_length=True, max_length=6144,truncation=True)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=4,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2,
                                  min_length=350,
                                  max_length=500)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    return summary
    
# Example usage
path = 'Case Links-20230417T110231Z-001/Case Links/'
file = open(path+'1947/urls_1947.txt','r')
df = pd.DataFrame(columns=['Case Number','Summary'])
count=0
for url in file:
    url = 'https://indiankanoon.org/doc/1072049/'
    summary = summarize_legal_document(url.strip())
    print(summary)
    break
#     case_num = url.split('/')[-2]
#     file = open(path+'1947/'+case_num+'.txt','w')
#     file.write(summary)
#     file.close()
#     df = pd.concat([pd.DataFrame([[case_num,summary]],columns=df.columns), df], ignore_index=True)
#     df.to_csv(path+'1947/summary_1947.csv',index=False)
#     count = count+1
#     if count%10 == 0:
#         print(count)


On May 26, 2018, the Honorable M.C. J. Krishnamoorthi Sastri of the Bombay High Court entered a final judgment against a former Commissioner of Income Tax for the city of Bombay.  According to the court's order, filed in the U.S. District Court for the Southern District of New York, on May 25, 1950, the former Commissioner raised the issue of whether municipal property taxes and urban immoveable property tax could be deducted as an allowable deduction under Section 9 of the Income-tax Act of 1940.  The court found that the municipal property tax and urban property tax were not allowable under the relevant provisions of the federal income-tax laws and instead were paid as a capital charge on the property.  In a parallel action, the Attorney General of India today announced that the court had entered a judgment that permanently enjoins the Commissioner from violating the antifraud provisions of Section 10(b) of the Securities Exchange Act of 1934 and Rules 10b-5(a) and (c) thereunder, an

In [38]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")

padding = "max_length"
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    document_length = len(text_content)
    if document_length < 1000:
        max_length = 100
    elif document_length < 2000:
        max_length = 150
    elif document_length < 3000:
        max_length = 200
    else:
        max_length = 250
    input_tokenized = tokenizer.encode(text_content, return_tensors='pt',padding=padding,pad_to_max_length=True, max_length=6144,truncation=True)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=4,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2,
                                  min_length=30,
                                  max_length=max_length)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    return summary
    
# Example usage
path = 'Case Links-20230417T110231Z-001/Case Links/'
file = open(path+'1947/urls_1947.txt','r')
df = pd.DataFrame(columns=['Case Number','Summary'])
count=0
for url in file:
    url = 'https://indiankanoon.org/doc/1072049/'
    summary = summarize_legal_document(url.strip())
    print(summary)
    break
#     case_num = url.split('/')[-2]
#     file = open(path+'1947/'+case_num+'.txt','w')
#     file.write(summary)
#     file.close()
#     df = pd.concat([pd.DataFrame([[case_num,summary]],columns=df.columns), df], ignore_index=True)
#     df.to_csv(path+'1947/summary_1947.csv',index=False)
#     count = count+1
#     if count%10 == 0:
#         print(count)


On May 26, 2018, the Honorable M.C. J. Krishnamoorthi Sastri of the Bombay High Court entered a final judgment against a former Commissioner of Income Tax for the city of Bombay.  According to the court's order, filed in the U.S. District Court for the Southern District of New York, on May 25, 1950, the former Commissioner raised the issue of whether municipal property taxes and urban immoveable property tax could be deducted as an allowable deduction under Section 9 of the Income-tax Act of 1940.  The court found that the municipal property tax and urban property tax were not allowable under the relevant provisions of the federal income-tax laws and instead were paid as a capital charge on the property.  In a parallel action, the Attorney General of India today announced that the court had entered a judgment that permanently enjoins the Commissioner from violating the antifraud provisions of Section 10(b) of the Securities Exchange Act of 1934 and Rules 10b-5(a) and (c) thereunder, an

We tried using nsi319/legal-led-base-16384 model which is trained specifically on legal documents 

# Using sumy library for summarization

In [4]:
pip install sumy


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21712 sha256=4d12665fd77730f885f6b69e232f0f534d9ace4c32a71cb34f4318ef833a15d5
  Stored in directory: c:\users\nsimh\appdata\local\pip\cache\wheels\ba\9f\70\7795228568b81b57a8932755938da9fb1f291b0576752604aa
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1871f4fd358686884b455963c51b47408973966f276915b59395f06e8bb09f64
  Stored in directory: c:\users\nsimh\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681896 sha256=dc870a32f

In [16]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words


LANGUAGE = "english"
SENTENCES_COUNT = 4


if __name__ == "__main__":
    url = "https://indiankanoon.org/doc/189173396/"
    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    # or for plain text files
    # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
    # parser = PlaintextParser.from_string("Check this out.", Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)

Mr. Mukul Rohtagi, learned Attorney General appearing for the Union of
India submitted that as the time schedule is fixed under Section 153(4)
for recovery is thirty days, by implication, the principle of issue of any
show cause notice is not attracted.
Assuming that the principles were
applicable to the case before us, in fact notices of personal hearing were
served on the petitioners by the Assistant Collector for a personal hearing
before the Assistant Collector passed the orders by which the petitioners
were held liable to repay the refunds made and to pay the excise on the
goods cleared for the [pic]subsequent periods.”
Relying on the same it is submitted by Mr. Rohatagi that as the
computation and the recovery are to be made within a time frame of thirty
days, issue of a show cause notice cannot be read into such a provision.
Ordinarily we would have adverted
to said submission advanced at the bar but we find, the assessee had not
demonstrably argued this ground and addressed the

# Sumy library is giving out the important sentences from the document which may not be necessarily the summary